In [20]:
from nlpsandboxclient import client
from copy import deepcopy

In [21]:
DATA_HOST = "http://localhost:8081/api/v1"
ANNO_HOST = "http://localhost:8080/api/v1"
ANNOTATION_STORE = 'covid'
DATASET_NAME = 'mayo-covid-dataset'


In [ ]:
datasets = client.list_datasets(host=DATA_HOST)

for dataset in datasets:
      print(dataset)

In [ ]:
example_note = {
   "identifier": "note-3",
   "type": "loinc:LP29684-5",
   "patientId": "507f1f77bcf86cd799439111",
   "text": "On 12/26/2020, Jake Li Kg, Mr.L coughed, Ms. Chloe Price met with 232323 Dr. Prescott id is 32243 in Seattle, WA, 206-434-3545, 343-2423343, 754-33-2343, jake@email.com, l232,1232 , Thai, wong, Linda, at Seattle, WA, hospital."
}

annotations = client.annotate_note(host=ANNO_HOST,
                            note=example_note,
                            tool_type="nlpsandbox:covid-symptom-annotator")

print(annotations)



In [ ]:
# option1 : annotate only 1 PHI category. In this example, date is annotated
type = 'covid-symptom'
types = ["covid-symptom"]
type_dict = {"covid-symptom":"textCovidSymptomAnnotations"}
notes = client.list_notes(host=DATA_HOST,
                  dataset_id=DATASET_NAME,
                  fhir_store_id="evaluation")

test_notes = list(notes)
print(len(test_notes))

annotation_list = []
converted_annotation_list = []
for i in test_notes:
    note_name= i.pop('note_name', None)
    note_id = note_name[-6:]
    print(note_id)
    # There is an error in annotation generated from  client.annotate_note, the note_name is "dataset/.."  but it should be "datasets/.."
    note_name = "datasets"+note_name[7:]
    converted_annotation = {"annotationSource":{"resourceSource":{"name":note_name}},"textContactAnnotations": [], "textCovidSymptomAnnotations":[],"textDateAnnotations":[],"textIdAnnotations":[],"textLocationAnnotations": [],"textPersonNameAnnotations": []}
    import time
    time.sleep(1)
    
    
    annotations = client.annotate_note(host=ANNO_HOST,
                            note=i,
                            tool_type=f"nlpsandbox:{type}-annotator")
    # original idType is other but it's not accepted, a quick fix to change 'other' 

    #for annotation in annotations["textIdAnnotations"]:
        #annotation['idType'] = 'account'
    #print(annotations[type_dict[type]])
    converted_annotation[type_dict[type]] = annotations[type_dict[type]]
    converted_annotation_list.append(converted_annotation)
    

In [ ]:
client.store_annotations(host=DATA_HOST,
                 dataset_id=DATASET_NAME,
                 annotation_store_id=ANNOTATION_STORE,
                 annotations=converted_annotation_list,
                 delete_existing_annotations = True)


In [ ]:
client.evaluate(host="http://localhost:8081/api/v1",
               dataset_id=DATASET_NAME,
               annotation_store_id=ANNOTATION_STORE,
               tool_type='nlpsandbox:covid-symptom-annotator')

In [ ]:
client.evaluate(host=DATA_HOST,
               dataset_id=DATASET_NAME,
               annotation_store_id=ANNOTATION_STORE,
               tool_type='nlpsandbox:covid-symptom-annotator')

In [ ]:
gold = client.list_annotations(host=DATA_HOST,
                  dataset_id=DATASET_NAME,
                  annotation_store_id="goldstandard")
gold = list(gold)
for i in gold:
    print(i["textCovidSymptomAnnotations"])